<a href="https://colab.research.google.com/github/buganart/BUGAN/blob/master/fid_score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@markdown Mount google drive.
from google.colab import output
from google.colab import drive
drive.mount('/content/drive')

# Check if we have linked the folder
from pathlib import Path
if not Path("/content/drive/My Drive/IRCMS_GAN_collaborative_database").exists():
    print(
        "Shortcut to our shared drive folder doesn't exits.\n\n"
        "\t1. Go to the google drive web UI\n"
        "\t2. Right click shared folder IRCMS_GAN_collaborative_database and click \"Add shortcut to Drive\""
    )

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


The FID score script is based on pytorch_fid package

In [2]:
#@title calculate FID score given 2 directories containing images
#@markdown Enter images of 3D object dataset location.  
#@markdown - For example via the file browser on the left to locate and right click to copy the path.)
#@markdown - file folder example: `/content/drive/My Drive/h/k` 
real_location = "/content/drive/My Drive/IRCMS_GAN_collaborative_database/Experiments/exportObjects/dataset_ply_out/img_folder" #@param {type:"string"}

#@markdown Enter images of generated 3D objects location.
#@markdown - For example via the file browser on the left to locate and right click to copy the path.)
#@markdown - file folder example: `/content/drive/My Drive/h/k`
generated_location = "/content/drive/My Drive/IRCMS_GAN_collaborative_database/Experiments/exportObjects/172mqt5l/img_folder" #@param {type:"string"}

print("real_location:", real_location)
print("generated_location:", generated_location)

real_location: /content/drive/My Drive/IRCMS_GAN_collaborative_database/Experiments/exportObjects/172mqt5l/img_folder
generated_location: /content/drive/My Drive/IRCMS_GAN_collaborative_database/Experiments/exportObjects/172mqt5l/img_folder


In [3]:
!pip install pytorch-fid
output.clear()
from pytorch_fid.fid_score import *
import torch

In [4]:
def create_model(device, dims):
    from pytorch_fid.inception import InceptionV3
    block_idx = InceptionV3.BLOCK_INDEX_BY_DIM[dims]
    model = InceptionV3([block_idx]).to(device)
    return model

In [ ]:
#reference: https://github.com/mseitzer/pytorch-fid/blob/master/pytorch_fid/fid_score.py

paths = [str(generated_location), str(real_location)]
batch_size = 64
device = torch.device('cuda' if (torch.cuda.is_available()) else 'cpu')
dims = 2048     #Dimensionality of Inception features to use. By default, uses pool3 features'

#calculate statistics for all paths
model = create_model(device, dims)
file_ext = ['.png', '.jpg']
for i in range(len(paths)):
    path = Path(paths[i])
    if path.suffix == ".npz":
        continue
    else:
        path_summary = path / "summary.npz"
    #check if summary file exists (if not it will create one)
    if not path_summary.exists():
        files = [
            file
            for file in path.rglob("*.*")
            if file.suffix in file_ext and not "__MACOSX" in str(file)
        ]
        m, s = calculate_activation_statistics(files, model, batch_size, dims, device)
        np.savez(str(path_summary), mu=m, sigma=s)
    #change path to summary file
    paths[i] = str(path_summary)

#calculate fid score
fid_value = calculate_fid_given_paths(paths, batch_size, device, dims)
print(fid_value)